In [1]:
# based on https://www.kaggle.com/piantic/how-to-finetuning-models-pytorch-xla-tpu

import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm

In [2]:
! ls ../input/cassava-disease

extraimages.zip  random.txt  sample_submission_file.csv  test.zip  train.zip


In [3]:
! unzip ../input/cassava-disease/train.zip > abc

In [4]:
! ls train

cbb  cbsd  cgm	cmd  healthy


In [5]:
import torch
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms
import numpy as np

In [6]:
transform = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor()
])

allimg = datasets.ImageFolder('./train', transform=transform)

validation_split = 0.25
dataset_size = len(allimg)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
np.random.seed(101)
np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

In [7]:
train_loader = torch.utils.data.DataLoader(allimg, batch_size=16, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(allimg, batch_size=16,
                                                sampler=valid_sampler)

In [8]:
for images, labels in train_loader:
    print(images[0])
    print(labels[0])
    break

tensor([[[0.4196, 0.4039, 0.3608,  ..., 0.5294, 0.5137, 0.4510],
         [0.4275, 0.4314, 0.4314,  ..., 0.4275, 0.3294, 0.2588],
         [0.4667, 0.4549, 0.4627,  ..., 0.2353, 0.2275, 0.2157],
         ...,
         [0.1804, 0.1765, 0.1725,  ..., 0.3216, 0.3412, 0.3725],
         [0.2471, 0.1961, 0.1725,  ..., 0.3529, 0.3961, 0.3882],
         [0.2980, 0.3020, 0.2510,  ..., 0.4039, 0.3529, 0.2863]],

        [[0.4157, 0.4118, 0.3686,  ..., 0.6275, 0.6118, 0.5529],
         [0.4314, 0.4431, 0.4431,  ..., 0.5333, 0.4392, 0.3725],
         [0.4745, 0.4667, 0.4745,  ..., 0.3412, 0.3373, 0.3373],
         ...,
         [0.3216, 0.3176, 0.3216,  ..., 0.5255, 0.5373, 0.5451],
         [0.3804, 0.3373, 0.3176,  ..., 0.5255, 0.5490, 0.5098],
         [0.4353, 0.4471, 0.4039,  ..., 0.5216, 0.4431, 0.3451]],

        [[0.3098, 0.2941, 0.2549,  ..., 0.4745, 0.4588, 0.3961],
         [0.3255, 0.3294, 0.3294,  ..., 0.3529, 0.2549, 0.1843],
         [0.3725, 0.3608, 0.3647,  ..., 0.1529, 0.1451, 0.

In [9]:
from torch import nn

class CustomResNext(nn.Module):
    def __init__(self, model_name, target_size, pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        n_features = self.model.fc.in_features
        self.model.fc = nn.Linear(n_features, target_size)

    def forward(self, x):
        x = self.model(x)
        return x
    
    def freeze(self):
        # To freeze the residual layers
        for param in self.model.parameters():
            param.requires_grad = False

        for param in self.model.fc.parameters():
            param.requires_grad = True
    
    def unfreeze(self):
        # Unfreeze all layers
        for param in self.model.parameters():
            param.requires_grad = True

In [10]:
allimg.class_to_idx

{'cbb': 0, 'cbsd': 1, 'cgm': 2, 'cmd': 3, 'healthy': 4}

In [11]:
m = CustomResNext('resnext50_32x4d', len(allimg.class_to_idx.keys()))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
m.to(device)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-rsb-weights/resnext50_32x4d_a1h-0146ab0a.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d_a1h-0146ab0a.pth


CustomResNext(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): ReLU(inplace=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_

In [12]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduce = reduce

    def forward(self, inputs, targets):
        BCE_loss = nn.CrossEntropyLoss()(inputs, targets)

        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss

In [13]:
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau

optimizer = Adam(filter(lambda p: p.requires_grad, m.parameters()), lr=1e-4, weight_decay=1e-6, amsgrad=False)
scheduler = ReduceLROnPlateau(optimizer, mode='min', verbose=True)
criterion = FocalLoss().to(device)
best_score = 0.
best_loss = np.inf
config_device = "GPU"

In [14]:
import math

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [15]:
from torch.cuda.amp import autocast, GradScaler

def train_fn(train_loader, model, criterion, optimizer, epoch, scheduler, device):
    scaler = GradScaler()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    # switch to train mode
    model.train()
    start = end = time.time()
    global_step = 0
    for step, (images, labels) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with autocast():
            y_preds = model(images)
            loss = criterion(y_preds, labels)
            # record loss
            losses.update(loss.item(), batch_size)
#             if CFG.gradient_accumulation_steps > 1:
#                 loss = loss / CFG.gradient_accumulation_steps
            scaler.scale(loss).backward()
            grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1000)
            if (step + 1) % 1 == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
                global_step += 1
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if step % 10 == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  #'LR: {lr:.6f}  '
                  .format(
                   epoch+1, step, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses,
                   remain=timeSince(start, float(step+1)/len(train_loader)),
                   grad_norm=grad_norm,
                   #lr=scheduler.get_lr()[0],
                   ))
    return losses.avg


def valid_fn(valid_loader, model, criterion, device):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    # switch to evaluation mode
    model.eval()
    trues = []
    preds = []
    start = end = time.time()
    for step, (images, labels) in enumerate(valid_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        # compute loss
        with torch.no_grad():
            y_preds = model(images)
        loss = criterion(y_preds, labels)
        losses.update(loss.item(), batch_size)
        # record accuracy
        trues.append(labels.to('cpu').numpy())
        preds.append(y_preds.softmax(1).to('cpu').numpy())
#         if CFG.gradient_accumulation_steps > 1:
#             loss = loss / CFG.gradient_accumulation_steps
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if step % 10 == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(
                   step, len(valid_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses,
                   remain=timeSince(start, float(step+1)/len(valid_loader)),
                   ))
    trues = np.concatenate(trues)
    predictions = np.concatenate(preds)
    return losses.avg, predictions, trues

In [16]:
from sklearn.metrics import accuracy_score
def get_score(y_true, y_pred):
    return accuracy_score(y_true, y_pred)

In [17]:
valid_labels = []
for img, labels in validation_loader:
    valid_labels += labels.tolist()
valid_labels[:20]

[3, 3, 3, 1, 1, 3, 1, 1, 1, 0, 3, 3, 0, 4, 0, 0, 3, 3, 0, 1]

In [18]:
import time
for epoch in range(2):

    start_time = time.time()

    avg_loss = train_fn(train_loader, m, criterion, optimizer, epoch, scheduler, device)
    avg_val_loss, preds, _ = valid_fn(validation_loader, m, criterion, device)

    score = get_score(valid_labels, preds.argmax(1))

    elapsed = time.time() - start_time

    print(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
    print(f'Epoch {epoch+1} - Score: {score:.4f}')
    
    if score > best_score:
        best_score = score
        print(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
        torch.save({'model': m.state_dict(), 
                        'preds': preds},
                       'best_score.pth')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:27: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.


Epoch: [1][0/266] Data 0.133 (0.133) Elapsed 0m 6s (remain 27m 31s) Loss: 1.0360(1.0360) Grad: nan  
Epoch: [1][10/266] Data 0.220 (0.151) Elapsed 0m 9s (remain 3m 40s) Loss: 0.9625(1.0009) Grad: 110231.9922  
Epoch: [1][20/266] Data 0.134 (0.147) Elapsed 0m 12s (remain 2m 28s) Loss: 0.9265(0.9637) Grad: 61095.6211  
Epoch: [1][30/266] Data 0.142 (0.146) Elapsed 0m 15s (remain 2m 0s) Loss: 0.8318(0.9233) Grad: 100498.1406  
Epoch: [1][40/266] Data 0.142 (0.145) Elapsed 0m 19s (remain 1m 44s) Loss: 0.6100(0.8755) Grad: 64602.4844  
Epoch: [1][50/266] Data 0.144 (0.148) Elapsed 0m 22s (remain 1m 34s) Loss: 0.7504(0.8357) Grad: 70926.4297  
Epoch: [1][60/266] Data 0.135 (0.147) Elapsed 0m 25s (remain 1m 25s) Loss: 0.4163(0.7989) Grad: 57372.5234  
Epoch: [1][70/266] Data 0.144 (0.146) Elapsed 0m 28s (remain 1m 18s) Loss: 0.3839(0.7641) Grad: 65638.7812  
Epoch: [1][80/266] Data 0.132 (0.147) Elapsed 0m 32s (remain 1m 13s) Loss: 0.5562(0.7306) Grad: 77030.4922  
Epoch: [1][90/266] Data 0.2

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:27: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.


Epoch: [2][0/266] Data 0.150 (0.150) Elapsed 0m 0s (remain 1m 25s) Loss: 0.1292(0.1292) Grad: inf  
Epoch: [2][10/266] Data 0.148 (0.145) Elapsed 0m 3s (remain 1m 20s) Loss: 0.3061(0.2348) Grad: 133694.8125  
Epoch: [2][20/266] Data 0.142 (0.143) Elapsed 0m 6s (remain 1m 17s) Loss: 0.0979(0.2195) Grad: 32636.3496  
Epoch: [2][30/266] Data 0.148 (0.142) Elapsed 0m 9s (remain 1m 14s) Loss: 0.0953(0.2338) Grad: 45237.3594  
Epoch: [2][40/266] Data 0.140 (0.144) Elapsed 0m 13s (remain 1m 11s) Loss: 0.0490(0.2287) Grad: 22865.5938  
Epoch: [2][50/266] Data 0.134 (0.143) Elapsed 0m 16s (remain 1m 8s) Loss: 0.2338(0.2109) Grad: 70679.2188  
Epoch: [2][60/266] Data 0.134 (0.143) Elapsed 0m 19s (remain 1m 5s) Loss: 0.1174(0.2026) Grad: 52915.1523  
Epoch: [2][70/266] Data 0.141 (0.145) Elapsed 0m 22s (remain 1m 2s) Loss: 0.0247(0.1922) Grad: 16509.8750  
Epoch: [2][80/266] Data 0.133 (0.149) Elapsed 0m 26s (remain 0m 59s) Loss: 0.0548(0.1840) Grad: 17126.5840  
Epoch: [2][90/266] Data 0.145 (0.

In [19]:
! unzip ../input/cassava-disease/test.zip > xyz

In [20]:
submission_img = datasets.ImageFolder('./test', transform=transform)
submission_loader = torch.utils.data.DataLoader(submission_img, batch_size=16)

In [21]:
filenames = []
for path, idx in submission_img.imgs:
    filenames.append(path[path.index('0/test') + 2:])
filenames[:20]

['test-img-0.jpg',
 'test-img-1.jpg',
 'test-img-10.jpg',
 'test-img-100.jpg',
 'test-img-1000.jpg',
 'test-img-1001.jpg',
 'test-img-1002.jpg',
 'test-img-1003.jpg',
 'test-img-1004.jpg',
 'test-img-1005.jpg',
 'test-img-1006.jpg',
 'test-img-1007.jpg',
 'test-img-1008.jpg',
 'test-img-1009.jpg',
 'test-img-101.jpg',
 'test-img-1010.jpg',
 'test-img-1011.jpg',
 'test-img-1012.jpg',
 'test-img-1013.jpg',
 'test-img-1014.jpg']

In [22]:
all_preds = []
for step, (images, labels) in enumerate(submission_loader):
    images = images.to(device)
    with torch.no_grad():
        all_preds += m(images).argmax(1).tolist()
all_preds[:20]

[1, 1, 1, 2, 1, 3, 1, 3, 3, 2, 1, 3, 4, 3, 3, 1, 3, 1, 3, 3]

In [23]:
! head -n 5 ../input/cassava-disease/sample_submission_file.csv

Category,Id
cbsd,test-img-0.jpg
cmd,test-img-1.jpg
cbb,test-img-2.jpg
cmd,test-img-3.jpg


In [24]:
rev_lookup = {}
for key in allimg.class_to_idx:
    rev_lookup[allimg.class_to_idx[key]] = key
rev_lookup

{0: 'cbb', 1: 'cbsd', 2: 'cgm', 3: 'cmd', 4: 'healthy'}

In [25]:
import csv
with open('submission.csv', 'w') as opfile:
    wrt = csv.writer(opfile)
    wrt.writerow(['Category', 'Id'])
    for idx in range(0, len(filenames)):
        wrt.writerow([
            rev_lookup[all_preds[idx]],
            filenames[idx]
        ])

In [26]:
! head -n 5 submission.csv